## Imports and Dataset Download

In [ ]:
%%capture
!pip install datasets
!pip install unsloth
# !pip install -U bitsandbytes

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import train_on_responses_only

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
%%capture
import pandas as pd
import io
from datasets import load_dataset

# Load SemEval 2025 task 8 Question-Answer splits
# semeval_train_qa = load_dataset("cardiffnlp/databench", name="semeval", split="train")
semeval_dev_qa = load_dataset("cardiffnlp/databench", name="semeval", split="dev")

In [ ]:
def get_model_tokenizer(model_name, max_seq_length=2048):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
        max_seq_length = max_seq_length,
        dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False.
        # trust_remote_code=True,
        # token = LLAMA_TOKEN, # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 16,
        lora_dropout = 0, # Supports any, but = 0 is optimized
        bias = "none",    # Supports any, but = "none" is optimized
        # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
        use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
        random_state = 3407,
        use_rslora = True,  # We support rank stabilized LoRA
        loftq_config = None, # And LoftQ
        max_seq_length=max_seq_length
    )

    return model, tokenizer

In [ ]:
model_name = "unsloth/phi-4-unsloth-bnb-4bit"

model, tokenizer = get_model_tokenizer(model_name)
FastLanguageModel.for_inference(model)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.39G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

...

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.15M [00:00<?, ?B/s]

Unsloth 2025.1.8 patched 40 layers with 40 QKV layers, 40 O layers and 40 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(100352, 5120, padding_idx=100351)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=5120, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4b

In [ ]:
def get_schema(df):
    # Capture DataFrame.info() output as a string
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()

    return str(info_str)

def generate_prompt(df, question):
    messages = [
        {"role": "system", "content":"""
        You are an expert Python data engineer.
    Your task is to generate pandas code based on a structured reasoning process
    You only generate code, no references or explanation - just code
    You generate only 20 lines of code at max

        """},
        {"role": "user", "content":f"""
    Dataframe Schema:
    {get_schema(df)}


    Sample Rows:
    {df.head(3).to_dict(orient='records')}

    User Question:
    {question}

    Expected Output Format:
    Generate runnable Python code that follows the given reasoning using pandas.
    The code should assume that the dataframe is already loaded as `df`.
    The final output should be stored in a variable named `result`.

    The expected answer type is unknown, but it will always be one of the following:
        * Boolean: True/False, "Y"/"N", "Yes"/"No" (case insensitive).
        * Category: A value from a cell (or substring of a cell) in the dataset.
        * Number: A numerical value from a cell or a computed statistic.
        * List[category]: A list of categories (unique or repeated based on context). Format: ['cat', 'dog'].
        * List[number]: A list of numbers.

    Given the user question, you need to write code in pandas, assume that you already have df.
    Generate only the code.
        """},
    ]
    # Given the user question, you need to reason out the steps (in bullet points) to write pandas code
    # Then write the code in pandas.
    return messages

In [ ]:
reason_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}
"""

In [ ]:
# reason_prompt.format(instruction="", input="", output="")
import torch
import json
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
df_outs = pd.DataFrame(columns = ['ind', 'question', 'llm_output', 'generated_code', 'result'])
semeval_test_qa = pd.read_csv('/content/competition/test_qa.csv')

In [ ]:
from tqdm import tqdm
from tqdm import trange

def execute_and_get_result(llm_output):
    # result = None
    generated_code = llm_output.split('```python')[-1].split("```")[0]
    # print(generated_code)

    try:
        exec(generated_code, globals())
        print('Execute code..')
    except Exception as E:
        return E  # Return the exception to be handled in chat retry

    # return str(global_scope["result"])

answers = []
result = None

correct_outs = 0
for ind in trange(len(semeval_test_qa)):
    fil = semeval_test_qa.iloc[ind, :]['dataset']
    df = pd.read_parquet(f'/content/competition/{fil}/sample.parquet')
    question = semeval_test_qa.iloc[ind, :]['question']

    prompt = generate_prompt(df, question)
    prompt_check = reason_prompt.format(instruction=str(prompt[0]['content']), input=str(prompt[1]['content']), output="")


    for _ in range(2):

        inputs = tokenizer([prompt_check], return_tensors = "pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
        llm_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split('### Response:\n')[-1]
        res = execute_and_get_result(llm_output)

        if isinstance(res, Exception):
            print(f"Retrying due to exception...{res}")

            # messages.append({'role':'assistant', 'content':llm_output})
            # messages.append({'role':'user', 'content':f"\n Error encountered: {str(res)}. Please fix the code and rewrite it again"})
            prompt_check += llm_output
            prompt_check += f"\n ### Code Feedback: \nError encountered: {str(res)}. Please fix the code and rewrite it again"
            prompt_check += f"\n ### Response:\n"

        else:
            break  # Exit loop if no exception

    # llm_output, ind
    # generated_code
    # result
    df_outs = pd.concat([df_outs, pd.DataFrame([{'ind':ind,
                                                 'question': question,
                                                 'llm_output':llm_output,
                                                 'generated_code':llm_output.split('```python')[-1].split("```")[0],
                                                 'result':result}])], ignore_index=True)

    answers.append(str(result))

    print('-'*150)
    print('Response final:')
    print(prompt_check.split('### Response:\n')[-1])


  0%|          | 1/522 [00:06<1:00:42,  6.99s/it]

Execute code..
------------------------------------------------------------------------------------------------------------------------------------------------------
Response final:




  0%|          | 2/522 [00:14<1:05:23,  7.55s/it]

Execute code..
------------------------------------------------------------------------------------------------------------------------------------------------------
Response final:




  1%|          | 3/522 [00:22<1:04:33,  7.46s/it]

...

Execute code..
------------------------------------------------------------------------------------------------------------------------------------------------------
Response final:




100%|█████████▉| 521/522 [1:46:49<00:06,  6.65s/it]

Execute code..
------------------------------------------------------------------------------------------------------------------------------------------------------
Response final:




100%|██████████| 522/522 [1:46:56<00:00, 12.29s/it]

Execute code..
------------------------------------------------------------------------------------------------------------------------------------------------------
Response final:




In [ ]:
df_outs.to_csv('kaudvk.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_outs.to_csv("/content/drive/MyDrive/Task8_results/df_out.csv")

## Final Submission

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture

%cd /content/drive/MyDrive/Task8

!gdown https://drive.google.com/file/d/1IpSi0gNPYj9a9lNbWPsL3TxIBILoLsfE/view --fuzzy
!unzip competition.zip
!rm -rf competition.zip
!rm -rf __MACOSX

In [ ]:
import pandas as pd
from tqdm import tqdm

results_df = pd.read_csv("Task8_Results_phi_4_unsloth_bnb_4bit.csv")[['question', 'generated_code', 'result']]
original_df = pd.read_csv("competition/test_qa.csv")

results_df = results_df.rename(columns={'result':'result_sample'})
results_df['dataset'] = original_df['dataset']

results_df = results_df[['question', 'dataset', 'generated_code', 'result_sample']]

results_df.head()

,question,dataset,generated_code,result_sample
0,Is our average employee older than 35?,066_IBM_HR,\nimport pandas as pd\n\n# Assuming df is alre...,True
1,Is the most frequent travel value rarely trave...,066_IBM_HR,\nresult = df['BusinessTravel'].value_counts()...,True
2,Is the highest DailyRate equal to 1499?,066_IBM_HR,\nimport pandas as pd\n\n# Assuming df is alre...,False
3,Is the highest DailyRate negative?,066_IBM_HR,\nresult = df['DailyRate'].max() < 0\n,False
4,Is our research dept bigger than sales?,066_IBM_HR,\n# Filter the dataframe for the Research & De...,True


In [ ]:
def execute_and_get_result(generated_code):
    try:
        exec(generated_code, globals())
        # print('Execute code..')
    except Exception as E:
        return E

In [ ]:
all_model_predictions = []

for _, row in tqdm(results_df.iterrows(), total=len(results_df)):
    result = None
    df = pd.read_parquet(f"competition/{row['dataset']}/all.parquet")
    _ = execute_and_get_result(row['generated_code'])
    all_model_predictions.append(str(result))

results_df['result_all'] = all_model_predictions

results_df.head()

  2%|▏         | 12/522 [00:00<00:16, 30.97it/s]<string>:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
 22%|██▏       | 116/522 [00:44<00:33, 12.10it/s]

True


 25%|██▍       | 128/522 [00:44<00:17, 22.11it/s]<string>:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
 54%|█████▍    | 282/522 [00:47<00:03, 62.53it/s]<string>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
<string>:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retai

True


 92%|█████████▏| 478/522 [02:42<04:18,  5.88s/it]<string>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 522/522 [02:43<00:00,  3.19it/s]


,question,dataset,generated_code,result_sample,result_all
0,Is our average employee older than 35?,066_IBM_HR,\nimport pandas as pd\n\n# Assuming df is alre...,True,True
1,Is the most frequent travel value rarely trave...,066_IBM_HR,\nresult = df['BusinessTravel'].value_counts()...,True,True
2,Is the highest DailyRate equal to 1499?,066_IBM_HR,\nimport pandas as pd\n\n# Assuming df is alre...,False,True
3,Is the highest DailyRate negative?,066_IBM_HR,\nresult = df['DailyRate'].max() < 0\n,False,False
4,Is our research dept bigger than sales?,066_IBM_HR,\n# Filter the dataframe for the Research & De...,True,True


In [ ]:
final_result_sample = [str(result).strip().replace('\n', '') for result in results_df['result_sample']]
final_result_all = [str(result).strip().replace('\n', '') for result in results_df['result_all']]

with open('predictions.txt', 'w') as f:
    f.write('\n'.join(final_result_all))
with open('predictions_lite.txt', 'w') as f:
    f.write('\n'.join(final_result_sample))
results_df.to_excel('submission_results.xlsx', index=False)

## Rest

In [ ]:
def execute_and_get_result(llm_output):
    # result = None
    generated_code = llm_output.split('```python')[-1].split("```")[0]
    # print(generated_code)

    try:
        exec(generated_code, globals())
        print('Execute code..')
    except Exception as E:
        return E  # Return the exception to be handled in chat retry

    # return str(global_scope["result"])

answers = []
result = None

correct_outs = 0
for ind in range(len(semeval_dev_qa)):

    example_data = semeval_dev_qa[ind]
    df = pd.read_parquet(f"/content/competition/{file}/sample.parquet")
    question = example_data['question']

    prompt = generate_prompt(df, question)
    prompt_check = reason_prompt.format(instruction=str(prompt[0]['content']), input=str(prompt[1]['content']), output="")


    for _ in range(2):

        inputs = tokenizer([prompt_check], return_tensors = "pt").to(device)
        outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True)
        llm_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split('### Response:\n')[-1]
        res = execute_and_get_result(llm_output)

        if isinstance(res, Exception):
            print(f"Retrying due to exception...{res}")

            # messages.append({'role':'assistant', 'content':llm_output})
            # messages.append({'role':'user', 'content':f"\n Error encountered: {str(res)}. Please fix the code and rewrite it again"})
            prompt_check += llm_output
            prompt_check += f"\n ### Code Feedback: \nError encountered: {str(res)}. Please fix the code and rewrite it again"
            prompt_check += f"\n ### Response:\n"

        else:
            break  # Exit loop if no exception

    # llm_output, ind
    # generated_code
    # result
    df_outs = pd.concat([df_outs, pd.DataFrame([{'ind':ind,
                                                 'llm_output':llm_output,
                                                 'generated_code':llm_output.split('```python')[-1].split("```")[0],
                                                 'result':result}])], ignore_index=True)

    answers.append(str(result))

    print('-'*150)
    print('Response final:')
    print(prompt_check.split('### Response:\n')[-1])

    gold = str(example_data['answer'])

    print('-'*150)
    if gold == answers[-1]:
        print('Correct')
        print('Answer', gold)
        correct_outs += 1
    else:
        print('Incorrect')
        print('Gold:', gold)
        print('Predicted:', result)
    print('-'*150)

    print('Fraction of correct Responses:')
    print(correct_outs/(ind+1))

    print('--'*50)

    break

In [ ]:
df_outs

,question,dataset
0,Is our average employee older than 35?,066_IBM_HR
1,Is the most frequent travel value rarely trave...,066_IBM_HR
2,Is the highest DailyRate equal to 1499?,066_IBM_HR
3,Is the highest DailyRate negative?,066_IBM_HR
4,Is our research dept bigger than sales?,066_IBM_HR
...,...,...
517,List the titles of books with fewer than 200 p...,080_Books
518,List the categories of books with ratings abov...,080_Books
519,List the authors of books with more than 10 re...,080_Books
520,List the 5 categories of the last five books i...,080_Books


In [ ]:
df_outs.to_csv("test_qa_predictions.csv", index=False)

In [ ]:
results_df = pd.DataFrame(model_predictions, columns=['llm_output', 'generated_code', 'result_sample'])
results_df = pd.concat([semeval_test_qa, results_df], axis=1)

pd.set_option('display.max_colwidth', None)
display(results_df)

df_outs.to_csv("test_qa_predictions.csv", index=False)

## Model

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import train_on_responses_only

import torch
from transformers import BitsAndBytesConfig
from datasets import load_dataset

import pandas as pd
import glob
from tqdm.auto import tqdm
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, AutoModel
import json
import re
import subprocess
import os
import gc
import io

In [ ]:
def get_model_tokenizer(model_name, max_seq_length=2048):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = model_name, # or choose "unsloth/Llama-3.2-1B-Instruct"
        max_seq_length = max_seq_length,
        dtype = None, # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
        load_in_4bit = True, # Use 4bit quantization to reduce memory usage. Can be False.
        # trust_remote_code=True,
        # token = LLAMA_TOKEN, # use one if using gated models like meta-llama/Llama-2-7b-hf
    )

    model = FastLanguageModel.get_peft_model(
        model,
        r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
        target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj",],
        lora_alpha = 16,
        lora_dropout = 0, # Supports any, but = 0 is optimized
        bias = "none",    # Supports any, but = "none" is optimized
        # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
        use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
        random_state = 3407,
        use_rslora = True,  # We support rank stabilized LoRA
        loftq_config = None, # And LoftQ
        max_seq_length=max_seq_length
    )

    return model, tokenizer

reasoning_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
code_model_name = "unsloth/codellama-7b-bnb-4bit"
device = 'cuda' if torch.cuda.is_available() else 'cpu'

reasoning_model, reasoning_tokenizer = get_model_tokenizer(reasoning_model_name)
code_model, code_tokenizer = get_model_tokenizer(code_model_name)

## Prompt

In [ ]:
reason_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}
Above is an instruction that describes a task on a pandas dataframe, paired with an input on which we want to perform the task. Reason out and write clear steps on how to achieve this in Python.
"""

code_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}
Above is an instruction that describes a task, paired with an input that provides further context. Write a code in python that appropriately completes the request.
"""

In [ ]:
def generate_reason_prompt(query, df):
    instruction =  """You are a helpful assistant that helps to break down problems into actionable steps.
You will be given a query and some context (in this case, a pandas dataframe with columns and a few rows).
Based on this, you will provide a structured step-by-step approach on how to solve the problem using pandas.

According to the expected answer types:
    * Boolean: Valid answers include True/False, Y/N, Yes/No (all case insensitive).
    * Category: A value from a cell (or a substring of a cell) in the dataset.
    * Number: A numerical value from a cell in the dataset, which may represent a computed statistic (e.g., average, maximum, minimum).
    * List[category]: A list containing a fixed number of categories. The expected format is: "['cat', 'dog']". Pay attention to the wording of the query to determine if uniqueness is required or if repeated values are allowed.
    * List[number]: Similar to List[category], but with numbers as its elements.
You will not know the specific type of answer expected, but you can be assured that it will be one of these types.
"""

    input = f"""Here is the query: "{query}"
The dataframe has the following columns: {', '.join(df.columns)}.
Here are a few rows of the dataframe: {df.head(3).to_dict(orient='records')}.

Reason and lay out the concise steps needed to write code using pandas/python to answer this query.
You should only give the steps, and not the code.
The instructions, along with the query and the sample dataframe should be sufficient to write the code in pandas"""

    output = ""

    return reason_prompt.format(instruction=instruction, input=input, output=output)

def generate_code_prompt(reasoning_steps, query, df):
    instruction = "You are a helpful assistant that writes pandas code based on the provided reasoning steps."

    input = f"""Here is the query: "{query}"
The dataframe has the following columns: {', '.join(df.columns)}.
Here are a few rows of the dataframe: {df.head(3).to_dict(orient='records')}.
Here are the steps to solve the problem:
```{reasoning_steps}```

Please write the corresponding pandas code in the provided blank
```python
# Write your code here
```
"""

    output = ""

    return code_prompt.format(instruction=instruction, input=input, output=output)

## Query LLM

In [ ]:
# Example usage
example_data = semeval_dev_qa[1]
df = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{example_data['dataset']}/sample.parquet")
question = example_data['question']

FastLanguageModel.for_inference(reasoning_model)
FastLanguageModel.for_inference(code_model)

# Step 1: Generate reasoning steps from the reasoning model
prompt = generate_reason_prompt(question, df)
inputs = reasoning_tokenizer([prompt], return_tensors = "pt").to(device)
outputs = reasoning_model.generate(**inputs, max_new_tokens = 512, use_cache = True)
reasoning_steps = reasoning_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split('### Response:\n')[-1]

# Step 2: Generate Code
prompt = generate_code_prompt(reasoning_steps, question, df)
inputs = code_tokenizer([prompt], return_tensors = "pt").to(device)
outputs = code_model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
generated_code = code_tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].split('Please write the corresponding pandas code in the provided blank\n')[-1]

# Print Example
print('Example:')
print(json.dumps(example_data, indent=4))
print('-'*150)

# Output the results
print("Reasoning Steps:")
print(reasoning_steps)
print('-'*150)

print("Generated Code:")
print(generated_code)
print('-'*150)

# Adarsh

In [ ]:
%%capture
# !pip install optimum auto-gptq
# !pip install --upgrade transformers
!pip install unsloth

In [ ]:
import json
import pandas as pd

from unsloth import FastLanguageModel

def get_model_tokenizer(model_name, max_seq_length=2048):
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=model_name,
        max_seq_length=max_seq_length,
        dtype=None,
        load_in_4bit=True,
        token=LLAMA_TOKEN
    )

    return model, tokenizer

reasoning_model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit"
LLAMA_TOKEN = 'ENTER_YOUR_TOKEN'

reasoning_model, reasoning_tokenizer = get_model_tokenizer(reasoning_model_name)
FastLanguageModel.for_inference(reasoning_model)

In [ ]:
reason_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Response:
{output}
"""

code_prompt = """### Instruction:
{instruction}

### Input:
{input}

### Solution:
{output}
"""

def get_schema(df):
    # Capture DataFrame.info() output as a string
    buffer = io.StringIO()
    df.info(buf=buffer)
    info_str = buffer.getvalue()

    return str(info_str)


def generate_reason_prompt(query, df):
    instruction = """You are an expert data analyst.
Your task is to analyze a question asked on a given pandas dataframe and break it down into a logical bullet-point approach for writing pandas code.
Do not generate the actual pandas code—just provide reasoning in short concise bullet points
"""

    input_text = f"""
Dataframe Schema:
{get_schema(df)}

Sample Rows:
{df.head(3).to_dict(orient='records')}

User Question:
{query}

Expected Output Format:
Provide a concise bullet-point explanation of how to retrieve the required answer using pandas.
"""
    print('*'*150)
    prompt = reason_prompt.format(instruction=instruction, input=input_text, output="")
    print(prompt)
    print('*'*150)
    return prompt

def generate_code_prompt(reasoning_steps, query, df):
    instruction = """
You are an expert Python data engineer.
Your task is to generate pandas code based on a structured reasoning process
You only generate code, no references or explanation - just code
You generate only 20 lines of code at max
"""

    input_text = f"""
Dataframe Schema:
{get_schema(df)}

Sample Rows:
{df.head(3).to_dict(orient='records')}

Step-by-Step Reasoning:
{reasoning_steps}

Expected Output Format:
Generate runnable Python code that follows the given reasoning using pandas.
The code should assume that the dataframe is already loaded as `df`.
The final output should be stored in a variable named `result`.

The expected answer type is unknown, but it will always be one of the following:
    * Boolean: True/False, "Y"/"N", "Yes"/"No" (case insensitive).
    * Category: A value from a cell (or substring of a cell) in the dataset.
    * Number: A numerical value from a cell or a computed statistic.
    * List[category]: A list of categories (unique or repeated based on context). Format: ['cat', 'dog'].
    * List[number]: A list of numbers.

Ensure the result is in the correct format based on the reasoning. If multiple valid formats exist, return the most relevant one based on the query."

Give only the code, nothing else.
"""
    print('*'*150)
    prompt = [
        {'role':'system', 'content':instruction},
        {'role':'user', 'content':input_text}
    ]
    prompt = code_prompt.format(instruction=instruction, input=input_text, output="")
    print(prompt)
    print('*'*150)
    return prompt

# def generate_review_prompt(generated_code, query, df):
#     instruction = f"""You are an AI that reviews pandas code for correctness and improvements.
# We only care about the accuracy of the code and alignment of the code to the steps.
# If you think the code looks good, then give the response phrase "Code is correct".
# Otherwise if the code is incorrect, then give the response phrase "Code is incorrect". """
#     input_text = f"""Query: "{query}"
# Columns: {', '.join(df.columns)}
# Sample Rows: {df.head(3).to_dict(orient='records')}
# Generated Code: ```python
# {generated_code}
# ```
# Provide a review and suggest refinements."""
#     return reason_prompt.format(instruction=instruction, input=input_text, output="")

def iterative_code_generation(query, df, max_iterations=1):

    # # Step 1: Generate reasoning steps
    # reason_prompt_text = generate_reason_prompt(query, df)
    # reason_inputs = reasoning_tokenizer([reason_prompt_text], return_tensors="pt").to(device)
    # reason_outputs = reasoning_model.generate(**reason_inputs, max_new_tokens=512, use_cache=True)
    # reasoning_steps = reasoning_tokenizer.batch_decode(reason_outputs, skip_special_tokens=True)[0].split('### Response:\n')[-1]

    reasoning_steps = 'a'
    print('-'*150)
    print("Reasoning Steps:")
    print(reasoning_steps)
    print('-'*150)

    generated_code = ""

    for _ in range(max_iterations):
        # Step 2: Generate initial code
        code_prompt_text = generate_code_prompt(reasoning_steps, query, df)
        # code_inputs = code_tokenizer([code_prompt_text], return_tensors="pt").to(device)
        # code_outputs = code_model.generate(**code_inputs, max_new_tokens=2048, use_cache=True)
        # generated_code = code_tokenizer.batch_decode(code_outputs, skip_special_tokens=True)[0].split('Response:\n')[-1]
        generated_code = code_llm_pipe(code_prompt_text, max_length=512)[0][-1]['content']

        # # Step 3: Review the generated code
        # review_prompt_text = generate_review_prompt(generated_code, query, df)
        # review_inputs = reasoning_tokenizer([review_prompt_text], return_tensors="pt").to(device)
        # review_outputs = reasoning_model.generate(**review_inputs, max_new_tokens=512, use_cache=True)
        # review_feedback = reasoning_tokenizer.batch_decode(review_outputs, skip_special_tokens=True)[0].split('Response:\n')[-1]

        # if "No improvements needed" in review_feedback or "Code is correct" in review_feedback:
        #     break  # Exit loop if the code is deemed correct

        # # Update reasoning steps based on feedback
        # reasoning_steps += f"\n\nFeedback from review: {review_feedback}"

        # print('-'*150)
        # print(f"\n\nFeedback from review: {review_feedback}")
        # print('-'*150)

    return reasoning_steps, generated_code

# Example usage
example_data = semeval_dev_qa[1]
df = pd.read_parquet(f"hf://datasets/cardiffnlp/databench/data/{example_data['dataset']}/sample.parquet")
question = example_data['question']

reasoning_steps, generated_code = iterative_code_generation(question, df)

# Print Example
print('Example:')
print(json.dumps(example_data, indent=4))
print('-'*150)



print("Generated Code:")
print(generated_code)
print('-'*150)